## Synthetic data

In [1]:
%load_ext autoreload 
%autoreload 2

import sys
import ast

import pandas as pd
import numpy as np

# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")
from src import aposteriori

import tasks.run_helper

In [2]:
syn_df = pd.read_csv(
    "../data/synthetic.csv",
    converters={
        "annot_personality_characteristics": ast.literal_eval,
        "Toxicity": ast.literal_eval,
        "Argument Quality": ast.literal_eval,
        "age_annot": ast.literal_eval,
        "sex_annot": ast.literal_eval,
        "sexual_orientation_annot": ast.literal_eval,
        "current_employment_annot": ast.literal_eval,
        "education_level_annot": ast.literal_eval,
    },
)
syn_df["comment_key"] = syn_df.message + syn_df.conv_id
syn_df["fake_index"] = 1

syn_df.Toxicity = syn_df.Toxicity.apply(lambda x: [int(tox) for tox in x])

syn_df.age_annot = syn_df.age_annot.apply(
    lambda ls: [int(x) for x in ls]
).apply(lambda x: pd.cut(x, bins=4))
syn_df

,conv_id,message,intent,Toxicity,Argument Quality,age_user,sex_user,sexual_orientation_user,demographic_group_user,current_employment_user,education_level_user,age_annot,sex_annot,sexual_orientation_annot,demographic_group_annot,current_employment_annot,education_level_annot,comment_key,fake_index
0,028f869d-cf12-444a-aefc-62f8b6294acd,@FitnessJunkie07 Respect and honesty are unive...,"['Neutral', 'Neutral', 'Neutral', 'Neutral', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1]","['25', '25', '25', '25', '25', '25', '25', '25...","['male', 'male', 'male', 'male', 'male', 'male...","['Bisexual', 'Bisexual', 'Bisexual', 'Bisexual...","['Asian', 'Asian', 'Asian', 'Asian', 'Asian', ...","['Software Developer', 'Software Developer', '...","[""Bachelor's"", ""Bachelor's"", ""Bachelor's"", ""Ba...","[(20.956, 32.0], (20.956, 32.0], (20.956, 32.0...","[female, male, male, male, male, male, female,...","[Lesbian, Pansexual, Homosexual, Heterosexual,...","['Arab', 'White', 'Caucasian', 'South Asian', ...","[Poet, Game Developer, Research Scientist, Ret...","[Master's, Some College, Bachelor's, PhD, Mast...",@FitnessJunkie07 Respect and honesty are unive...,1
1,028f869d-cf12-444a-aefc-62f8b6294acd,"@FitnessJunkie07, I agree, but sometimes chang...","['Neutral', 'Neutral', 'Neutral', 'Neutral', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","['27', '27', '27', '27', '27', '27', '27', '27...","['female', 'female', 'female', 'female', 'fema...","['Lesbian', 'Lesbian', 'Lesbian', 'Lesbian', '...","['Arab', 'Arab', 'Arab', 'Arab', 'Arab', 'Arab...","['Poet', 'Poet', 'Poet', 'Poet', 'Poet', 'Poet...","[""Master's"", ""Master's"", ""Master's"", ""Master's...","[(20.956, 32.0], (20.956, 32.0], (20.956, 32.0...","[female, male, male, male, male, male, female,...","[Lesbian, Pansexual, Homosexual, Heterosexual,...","['Arab', 'White', 'Caucasian', 'South Asian', ...","[Poet, Game Developer, Research Scientist, Ret...","[Master's, Some College, Bachelor's, PhD, Mast...","@FitnessJunkie07, I agree, but sometimes chang...",1
2,028f869d-cf12-444a-aefc-62f8b6294acd,"@FitnessJunkie07, I completely agree that resp...","['Community-oriented', 'Community-oriented', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2]","['58', '58', '58', '58', '58', '58', '58', '58...","['female', 'female', 'female', 'female', 'fema...","['Heterosexual', 'Heterosexual', 'Heterosexual...","['Hispanic', 'Hispanic', 'Hispanic', 'Hispanic...","['Nonprofit Organizer', 'Nonprofit Organizer',...","[""Bachelor's"", ""Bachelor's"", ""Bachelor's"", ""Ba...","[(20.956, 32.0], (20.956, 32.0], (20.956, 32.0...","[female, male, male, male, male, male, female,...","[Lesbian, Pansexual, Homosexual, Heterosexual,...","['Arab', 'White', 'Caucasian', 'South Asian', ...","[Poet, Game Developer, Research Scientist, Ret...","[Master's, Some College, Bachelor's, PhD, Mast...","@FitnessJunkie07, I completely agree that resp...",1
3,028f869d-cf12-444a-aefc-62f8b6294acd,"@FitnessJunkie07, I understand where you're co...","['Community-oriented', 'Community-oriented', '...","[1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2]","[2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2]","['58', '58', '58', '58', '58', '58', '58', '58...","['female', 'female', 'female', 'female', 'fema...","['Heterosexual', 'Heterosexual', 'Heterosexual...","['Hispanic', 'Hispanic', 'Hispanic', 'Hispanic...","['Nonprofit Organizer', 'Nonprofit Organizer',...","[""Bachelor's"", ""Bachelor's"", ""Bachelor's"", ""Ba...","[(20.956, 32.0], (20.956, 32.0], (20.956, 32.0...","[female, male, male, male, male, male, female,...","[Lesbian, Pansexual, Homosexual, Heterosexual,...","['Arab', 'White', 'Caucasian', 'South Asian', ...","[Poet, Game Developer, Research Scientist, Ret...","[Master's, Some College, Bachelor's, PhD, Mast...","@FitnessJunkie07, I understand where you're co...",1
4,028f869d-cf12-444a-aefc-62f8b6294acd,@LoyalVolunteer54 @FitnessJunkie07 It's import...,"['Moderator', '

In [3]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="conv_id",
    sdb_column="age_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/140 [00:00<?, ?it/s]

,028f869d-cf12-444a-aefc-62f8b6294acd,0594f5fe-3bef-4642-8bb7-9ac356f44acb,05d53124-194a-4608-9614-c7075f71a827,07bd0dbc-5591-4b1a-ba75-fb7a84c28986,095f8e2c-4c6b-45d3-983b-c0a836cd1d4e,0d4caeec-0d56-4074-a2df-4bf5012d1c19,11d555f4-f5e4-4f64-a0cd-f89f2b29553d,1257a767-cba5-43c7-a532-deda418022e5,139dce53-526b-4950-83cc-1ce060739e3e,15f0ac31-1302-40eb-ba03-b486561cfd2b,...,ed709c3a-29f6-45f3-a63b-f1f4283d3d89,f4c92f48-1045-49db-9ad5-c5aee0f650cc,f66c6907-aa30-4dc3-8752-fa4b6d926e14,f7221fe4-3eac-42f2-937a-1dacbbe72395,f8085173-8cbf-4c15-a9e8-713c40539bdd,f83f95e7-1200-4dcd-a620-29694858190b,fac8059e-390f-421a-bbb8-a4b615da8d35,fce13b53-ae50-47b2-b70e-1de66f31a8bf,fdb670f7-dd7e-4222-8f5e-e3c6a8fccff4,fe858614-6571-43e7-badf-3cdcb38337b3
"(43.0, 54.0]",-0.099279,0.014374,0.017857,0.108048,-0.007557,0.069594,0.009901,0.127400,-0.040150,-0.059495,...,-0.039376,0.020568,-0.036941,0.037254,-0.016220,-0.068702,-0.090729,-0.020608,-0.109513,0.172794
"(32.0, 43.0]",-0.116149,-0.029230,-0.059603,-0.134062,-0.005025,-0.045361,0.012346,-0.062417,-0.086048,0.020208,...,-0.122244,0.017682,-0.079914,0.031579,-0.017613,-0.032505,-0.076321,0.023247,-0.062526,-0.025163
"(54.0, 65.0]",0.021097,-0.001703,-0.035840,-0.032132,0.015897,-0.004474,-0.009346,0.099099,-0.006612,0.039027,...,0.020549,0.007539,0.015604,-0.076340,-0.035673,-0.036653,0.021352,0.027274,0.073983,-0.059098
"(20.956, 32.0]",-0.004452,0.008792,0.061104,0.022812,0.065749,0.010452,0.004191,-0.027397,0.014368,-0.022263,...,-0.007356,0.023682,0.023409,0.064611,0.031959,-0.001585,-0.089963,-0.007024,-0.015572,0.046413


In [4]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="conv_id",
    sdb_column="sexual_orientation_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/140 [00:00<?, ?it/s]

,028f869d-cf12-444a-aefc-62f8b6294acd,0594f5fe-3bef-4642-8bb7-9ac356f44acb,05d53124-194a-4608-9614-c7075f71a827,07bd0dbc-5591-4b1a-ba75-fb7a84c28986,095f8e2c-4c6b-45d3-983b-c0a836cd1d4e,0d4caeec-0d56-4074-a2df-4bf5012d1c19,11d555f4-f5e4-4f64-a0cd-f89f2b29553d,1257a767-cba5-43c7-a532-deda418022e5,139dce53-526b-4950-83cc-1ce060739e3e,15f0ac31-1302-40eb-ba03-b486561cfd2b,...,ed709c3a-29f6-45f3-a63b-f1f4283d3d89,f4c92f48-1045-49db-9ad5-c5aee0f650cc,f66c6907-aa30-4dc3-8752-fa4b6d926e14,f7221fe4-3eac-42f2-937a-1dacbbe72395,f8085173-8cbf-4c15-a9e8-713c40539bdd,f83f95e7-1200-4dcd-a620-29694858190b,fac8059e-390f-421a-bbb8-a4b615da8d35,fce13b53-ae50-47b2-b70e-1de66f31a8bf,fdb670f7-dd7e-4222-8f5e-e3c6a8fccff4,fe858614-6571-43e7-badf-3cdcb38337b3
Homosexual,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
Bisexual,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
Lesbian,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
Asexual,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
Heterosexual,-0.026338,-0.028906,-0.133953,-0.014993,-0.029673,-0.014271,0.002178,0.040562,0.044781,-0.021802,...,-0.063869,0.016814,-0.010057,0.008677,-0.009799,-0.05806,-0.039316,-0.000868,-0.007875,0.024227
Pansexual,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000


In [5]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="conv_id",
    sdb_column="education_level_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/140 [00:00<?, ?it/s]

,028f869d-cf12-444a-aefc-62f8b6294acd,0594f5fe-3bef-4642-8bb7-9ac356f44acb,05d53124-194a-4608-9614-c7075f71a827,07bd0dbc-5591-4b1a-ba75-fb7a84c28986,095f8e2c-4c6b-45d3-983b-c0a836cd1d4e,0d4caeec-0d56-4074-a2df-4bf5012d1c19,11d555f4-f5e4-4f64-a0cd-f89f2b29553d,1257a767-cba5-43c7-a532-deda418022e5,139dce53-526b-4950-83cc-1ce060739e3e,15f0ac31-1302-40eb-ba03-b486561cfd2b,...,ed709c3a-29f6-45f3-a63b-f1f4283d3d89,f4c92f48-1045-49db-9ad5-c5aee0f650cc,f66c6907-aa30-4dc3-8752-fa4b6d926e14,f7221fe4-3eac-42f2-937a-1dacbbe72395,f8085173-8cbf-4c15-a9e8-713c40539bdd,f83f95e7-1200-4dcd-a620-29694858190b,fac8059e-390f-421a-bbb8-a4b615da8d35,fce13b53-ae50-47b2-b70e-1de66f31a8bf,fdb670f7-dd7e-4222-8f5e-e3c6a8fccff4,fe858614-6571-43e7-badf-3cdcb38337b3
Bachelor's,-0.036986,0.016595,0.117387,-0.012966,-0.088012,-0.005587,0.047256,-0.071429,-0.002297,0.018405,...,-0.001603,0.016522,-0.086012,-0.098205,0.016522,0.049610,-0.004520,-0.018809,-0.078614,0.085087
No formal education,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
PhD,0.100346,0.014778,-0.073826,-0.045016,-0.010101,-0.007839,-0.033097,0.290466,-0.004604,0.097991,...,0.034400,0.093775,0.056199,-0.146010,-0.059239,-0.031322,-0.189296,0.095734,0.135447,-0.006404
Some College,0.095652,-0.026905,0.007220,-0.093560,0.030820,-0.081283,0.005356,-0.030928,0.044652,-0.059495,...,0.040576,0.048563,0.005620,0.029331,0.057900,0.048706,-0.087494,0.012638,-0.033485,0.165121
Master's,-0.002735,0.055225,-0.167109,0.007576,0.013674,0.015022,0.025625,-0.028865,0.102258,0.030286,...,-0.009229,-0.006689,0.039013,0.046410,0.023875,-0.002793,0.217923,-0.005798,0.076840,-0.000441


In [6]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="conv_id",
    sdb_column="current_employment_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/140 [00:00<?, ?it/s]

,028f869d-cf12-444a-aefc-62f8b6294acd,0594f5fe-3bef-4642-8bb7-9ac356f44acb,05d53124-194a-4608-9614-c7075f71a827,07bd0dbc-5591-4b1a-ba75-fb7a84c28986,095f8e2c-4c6b-45d3-983b-c0a836cd1d4e,0d4caeec-0d56-4074-a2df-4bf5012d1c19,11d555f4-f5e4-4f64-a0cd-f89f2b29553d,1257a767-cba5-43c7-a532-deda418022e5,139dce53-526b-4950-83cc-1ce060739e3e,15f0ac31-1302-40eb-ba03-b486561cfd2b,...,ed709c3a-29f6-45f3-a63b-f1f4283d3d89,f4c92f48-1045-49db-9ad5-c5aee0f650cc,f66c6907-aa30-4dc3-8752-fa4b6d926e14,f7221fe4-3eac-42f2-937a-1dacbbe72395,f8085173-8cbf-4c15-a9e8-713c40539bdd,f83f95e7-1200-4dcd-a620-29694858190b,fac8059e-390f-421a-bbb8-a4b615da8d35,fce13b53-ae50-47b2-b70e-1de66f31a8bf,fdb670f7-dd7e-4222-8f5e-e3c6a8fccff4,fe858614-6571-43e7-badf-3cdcb38337b3
Game Developer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Botanist,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Registered Nurse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Research Scientist,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Farmer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Poet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Stock Trader,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Historian,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Travel Blogger,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cybersecurity Expert,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="conv_id",
    sdb_column="sex_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/140 [00:00<?, ?it/s]

,028f869d-cf12-444a-aefc-62f8b6294acd,0594f5fe-3bef-4642-8bb7-9ac356f44acb,05d53124-194a-4608-9614-c7075f71a827,07bd0dbc-5591-4b1a-ba75-fb7a84c28986,095f8e2c-4c6b-45d3-983b-c0a836cd1d4e,0d4caeec-0d56-4074-a2df-4bf5012d1c19,11d555f4-f5e4-4f64-a0cd-f89f2b29553d,1257a767-cba5-43c7-a532-deda418022e5,139dce53-526b-4950-83cc-1ce060739e3e,15f0ac31-1302-40eb-ba03-b486561cfd2b,...,ed709c3a-29f6-45f3-a63b-f1f4283d3d89,f4c92f48-1045-49db-9ad5-c5aee0f650cc,f66c6907-aa30-4dc3-8752-fa4b6d926e14,f7221fe4-3eac-42f2-937a-1dacbbe72395,f8085173-8cbf-4c15-a9e8-713c40539bdd,f83f95e7-1200-4dcd-a620-29694858190b,fac8059e-390f-421a-bbb8-a4b615da8d35,fce13b53-ae50-47b2-b70e-1de66f31a8bf,fdb670f7-dd7e-4222-8f5e-e3c6a8fccff4,fe858614-6571-43e7-badf-3cdcb38337b3
non-binary,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
female,-0.028924,-0.023347,-0.054932,-0.032668,0.040829,-0.004277,-0.006828,0.053356,-0.052041,-0.039529,...,-0.074377,0.034711,0.013581,0.019980,-0.031814,0.008201,-0.163824,-0.016039,-0.082032,-0.044075
male,0.006735,-0.015624,0.125164,-0.003891,-0.033055,-0.018607,-0.040819,-0.076003,-0.032427,0.012861,...,0.031425,-0.025929,0.005511,-0.020959,-0.049649,-0.011965,-0.039099,-0.023508,0.032416,0.018262


## Unified dataset

In [8]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="fake_index",
    sdb_column="age_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/1 [00:00<?, ?it/s]

,1
"(43.0, 54.0]",-0.002168
"(32.0, 43.0]",-0.007829
"(54.0, 65.0]",0.005366
"(20.956, 32.0]",-0.003430


In [9]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="fake_index",
    sdb_column="sexual_orientation_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/1 [00:00<?, ?it/s]

,1
Homosexual,0.000000
Bisexual,0.000000
Lesbian,0.000000
Asexual,0.000000
Heterosexual,0.004172
Pansexual,0.000000


In [10]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="fake_index",
    sdb_column="education_level_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/1 [00:00<?, ?it/s]

,1
Bachelor's,-0.002209
No formal education,0.000000
PhD,0.006734
Some College,-0.007761
Master's,0.003052


In [11]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="fake_index",
    sdb_column="current_employment_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/1 [00:00<?, ?it/s]

,1
Game Developer,0.0
Botanist,0.0
Registered Nurse,0.0
Research Scientist,0.0
Farmer,0.0
Poet,0.0
Stock Trader,0.0
Historian,0.0
Travel Blogger,0.0
Cybersecurity Expert,0.0


In [12]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="fake_index",
    sdb_column="sex_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/1 [00:00<?, ?it/s]

,1
non-binary,0.000000
female,-0.006157
male,-0.000671
